We have a list of artist names identified with a unique key in one dataset.

We have a dataset of with many more artists. We want to identify the subset of artists in this dataset that are also in the dataset above. Yet there may be some variation in name spelling.

We will use 'fuzzywuzzy' to do some fuzzy string matching. 

In [1]:
import pandas as pd
UKData = pd.read_csv("Other_files/ClaireDupin_Data.csv", sep=';')
SalonData = pd.read_csv("Other_files/Salons1848-1880_Data.csv" , sep=';')

In [74]:
UKData.head()

,UK_Omeka,dcterms:creator,dcterms:abstract
0,33,"Achenbach, Oswald","Düsseldorf, 1827-Düsseldorf, 1905"
1,34,"Adan, Louis-Émile","Paris, 1839-?, 1937"
2,35,"Alma Tadema, Lawrence","Dronrijp, 1836-?, 1902"
3,36,"Anastasi, Auguste","Paris, 1820-Paris, 1889"
4,37,"Anker, Albert","Anet, 1831-Berne, 1910"


In [75]:
SalonData.head()

,Exposant,Titre,Année,Salon
0,"Aarestrup, Mlle Marie","Portrait de Mlle Nilsson, artiste du Théâtre-L...",1866,Salon de 1866
1,"Aaron, Michel-Joseph",*Giroflées,1878,Salon de 1878
2,"Abbat, Auguste",Pauvre hirondelle,1880,Salon de 1880
3,"Abbema, Mlle Louise",*La duchesse Josiane,1875,Salon de 1875
4,"Abbema, Mlle Louise",*Le déjeûner dans la serre,1877,Salon de 1877


We would like to find the artists in the SalonData file that are also present in the file above, and assign them the corresponding unique key.  First, we'll create a column for this unique key. We'll also create a column to store the matched text. 

In [2]:
SalonData['UniqueKey']=''

In [3]:
SalonData['Match']=''

In [4]:
SalonData['Certainty']=''

In [153]:
SalonData.head()

,Exposant,Titre,Année,Salon,UniqueKey,Match,Certainty
0,"Aarestrup, Mlle Marie","Portrait de Mlle Nilsson, artiste du Théâtre-L...",1866,Salon de 1866,,,
1,"Aaron, Michel-Joseph",*Giroflées,1878,Salon de 1878,,,
2,"Abbat, Auguste",Pauvre hirondelle,1880,Salon de 1880,,,
3,"Abbema, Mlle Louise",*La duchesse Josiane,1875,Salon de 1875,,,
4,"Abbema, Mlle Louise",*Le déjeûner dans la serre,1877,Salon de 1877,,,


From the other data set, we will create a unique list of UniqueKey- Name pairs:

In [60]:
UKDict = {}
NameDict ={}

for index, row in UKData.iterrows():
    a = row['UK_Omeka']
    if a not in UKDict:
        b= row['dcterms:creator']
        UKDict.update({a:b}) 
        NameDict.update({b:a})

In [76]:
print(UKDict)

{33: 'Achenbach, Oswald', 34: 'Adan, Louis-Émile', 35: 'Alma Tadema, Lawrence', 36: 'Anastasi, Auguste', 37: 'Anker, Albert', 38: 'Antigna, Alexandre', 39: 'Appian, Adolphe', 40: 'Armand-Dumaresq, Édouard', 41: 'Aubert, Jean', 42: 'Aublet, Albert', 43: 'Auguin, Louis-Auguste', 44: 'Aze, Adolphe', 45: 'Baader, Louis-Marie', 46: 'Badin, Jules', 47: 'Ballavoine, Jules', 48: 'Balleroy, Albert de', 49: 'Balze, Paul', 50: 'Barillot, Léon', 51: 'Barrias, Félix', 52: 'Bastien-Lepage, Jules', 53: 'Baudit, Amédée', 54: 'Baudry, Paul', 55: 'Beaucé, Jean-Adolphe', 56: 'Beaulieu, Anatole de', 57: 'Beaumetz, Étienne', 58: 'Beaumont, Édouard de', 59: 'Beauverie, Charles', 60: 'Becker, Georges', 61: 'Bellanger, Camille', 62: 'Bellay, Charles-Paul', 63: 'Belly, Léon', 64: 'Benner, Jean', 65: 'Bénouville, François-Léon', 66: 'Berchère, Narcisse', 67: 'Bergeret, Denis', 68: 'Berne-Bellecour, Étienne', 69: 'Bernier, Camille', 70: 'Berthon, Nicolas', 71: 'Bertrand, James', 72: 'Besnard, Louise', 73: 'Besna

First, the easy stuff. We will match any names that are exactly the same.

In [199]:
for index, row in SalonData.iterrows():
    a= row['Exposant']
    if a in UKNameDict.keys():
        b= UKNameDict[a]
        SalonData.at[index,'UniqueKey'] = b
        SalonData.at[index,'Match'] = a
        SalonData.at[index,'Certainty'] = 100

In [185]:
SalonData.groupby(['Certainty']).count()

,Exposant,Titre,Année,Salon,UniqueKey,Match
Certainty,,,,,,
100,9056,9056,9056,9056,9056,9056
,56388,56388,56388,56388,56388,56388


Next, we import the FuzzyWuzzy module, a module for fuzzy string matching.

In [43]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [200]:
UKAdditions = {}

In [201]:
# First, search for exact match of updated list
for index, row in SalonData.iterrows():
    a= row['Exposant']
# If exact match does not exist, search for fuzzy match >90:
    if row['UniqueKey']=='': 
        for key, value in UKNameDict.items():
            if fuzz.token_sort_ratio(row['Exposant'],key) > 90:
                b= UKNameDict.get(key)
                SalonData.at[index,'UniqueKey'] = b
                SalonData.at[index,'Match'] = a 
                SalonData.at[index,'Certainty'] = 90
                UKAdditions.update({row['Exposant']:row['UniqueKey']})
                break
                    
# First, search for exact match of updated list
for index, row in SalonData.iterrows():
    a= row['Exposant']
    #Only in rows without a unique Key
    if row['UniqueKey']=='':   
        for key, value in UKNameDict.items():
            if fuzz.token_sort_ratio(row['Exposant'],key) > 80:
                b= UKNameDict.get(key)
                SalonData.at[index,'UniqueKey'] = b
                SalonData.at[index,'Match'] = a 
                SalonData.at[index,'Certainty'] = 80
                UKAdditions.update({row['Exposant']:row['UniqueKey']})
                break

# First, search for exact match of updated list
for index, row in SalonData.iterrows():
    a= row['Exposant']
    #Only in rows without a unique Key
    if row['UniqueKey']=='':
        for key, value in UKNameDict.items():
            if fuzz.token_sort_ratio(row['Exposant'],key) > 70:
                b= UKNameDict.get(key)
                SalonData.at[index,'UniqueKey'] = b
                SalonData.at[index,'Match'] = a 
                SalonData.at[index,'Certainty'] = 70
                UKAdditions.update({row['Exposant']:row['UniqueKey']})
                break

In [205]:
SalonData.groupby(['UniqueKey']).count()

,Exposant,Titre,Année,Salon,Match,Certainty
UniqueKey,,,,,,
33,26,26,26,26,26,26
34,80,80,80,80,80,80
35,14,14,14,14,14,14
36,64,64,64,64,64,64
37,91,91,91,91,91,91
...,...,...,...,...,...,...
648,49,49,49,49,49,49
649,19,19,19,19,19,19
650,11,11,11,11,11,11


In [223]:
UniqueMatches = SalonData.loc[SalonData['UniqueKey']!='',:]

In [224]:
UniqueMatches = UniqueMatches[['UniqueKey','Exposant','Certainty']]

In [226]:
UniqueMatches

,UniqueKey,Exposant,Certainty
2,36,"Abbat, Auguste",70
3,34,"Abbema, Mlle Louise",70
4,34,"Abbema, Mlle Louise",70
71,134,"Accard, Eugène",70
102,134,"Accard, Eugène",70
...,...,...,...
65317,367,"Zimmermann, Albert",70
65344,569,"Zo, Achille",100
65377,570,"Zuber, Jean-Henri",100
65385,570,"Zuber, Jean-Henri",90


In [225]:
UniqueMatches = UniqueMatches.drop_duplicates()

In [248]:
Stringlist =pd.DataFrame(columns=['Key','ClaireName','Match','Certainty'])

for index,row in UniqueMatches.iterrows():
    c = UniqueMatches.loc[index,'Exposant']
    d = UniqueMatches.loc[index,'Certainty']
    b = int(UniqueMatches.loc[index,'UniqueKey'])
    for key, value in UKNameDict.items(): 
         if value == b: 
            a = key
            Stringlist=Stringlist.append({'Key':[b],'ClaireName':[a],'Match':[c],'Certainty':[d]}, ignore_index=True)

In [253]:
Stringlist.to_csv('MatchDictionary.csv',',')

In [252]:
Stringlist.sort_values('Certainty')

,Key,ClaireName,Match,Certainty
0,[36],"[Anastasi, Auguste]","[Abbat, Auguste]",[70]
2792,[604],"[Gudin, Théodore]","[Lejeune, Théodore]",[70]
2793,[616],"[Lehmann, Henri]","[Leland, Henry ]",[70]
2794,[616],"[Lehmann, Henri]","[Leland, Henry]",[70]
2799,[178],"[Delort, Charles]","[Lelièvre, Charles]",[70]
...,...,...,...,...
1098,[155],"[Courtat, Louis]","[Courtat, Louis]",[100]
266,[60],"[Becker, Georges]","[Becker, Georges]",[100]
2986,[364],"[Machard, Jules]","[Machard, Jules]",[100]
4261,[511],"[Soyer, Paul]","[Soyer, Paul]",[100]


In [ ]:
First round with family names, second round with given name

In [53]:
UKData2 = pd.read_csv("Other_files/ClaireDupin_Data.csv", sep=';')
SalonData2 = pd.read_csv("Other_files/Salons1848-1880_Data.csv" , sep=';')
SalonData2['UniqueKey']=''
SalonData2['Match']=''
SalonData2['Certainty']=''

In [62]:
UKData2.head()

,UK_Omeka,dcterms:creator,dcterms:abstract
0,33,"Achenbach, Oswald","Düsseldorf, 1827-Düsseldorf, 1905"
1,34,"Adan, Louis-Émile","Paris, 1839-?, 1937"
2,35,"Alma Tadema, Lawrence","Dronrijp, 1836-?, 1902"
3,36,"Anastasi, Auguste","Paris, 1820-Paris, 1889"
4,37,"Anker, Albert","Anet, 1831-Berne, 1910"


In [63]:
SalonData2.head()

,Exposant,Titre,Année,Salon,UniqueKey,Match,Certainty
0,"Aarestrup, Mlle Marie","Portrait de Mlle Nilsson, artiste du Théâtre-L...",1866,Salon de 1866,,,
1,"Aaron, Michel-Joseph",*Giroflées,1878,Salon de 1878,,,
2,"Abbat, Auguste",Pauvre hirondelle,1880,Salon de 1880,,,
3,"Abbema, Mlle Louise",*La duchesse Josiane,1875,Salon de 1875,,,
4,"Abbema, Mlle Louise",*Le déjeûner dans la serre,1877,Salon de 1877,,,


In both datasets, let's split the columns into las name, first name

In [19]:
UKData2[['First','Last']] = UKData2['dcterms:creator'].str.split(",",expand=True)

In [20]:
UKData2.head()

,UK_Omeka,dcterms:creator,dcterms:abstract,First,Last
0,33,"Achenbach, Oswald","Düsseldorf, 1827-Düsseldorf, 1905",Achenbach,Oswald
1,34,"Adan, Louis-Émile","Paris, 1839-?, 1937",Adan,Louis-Émile
2,35,"Alma Tadema, Lawrence","Dronrijp, 1836-?, 1902",Alma Tadema,Lawrence
3,36,"Anastasi, Auguste","Paris, 1820-Paris, 1889",Anastasi,Auguste
4,37,"Anker, Albert","Anet, 1831-Berne, 1910",Anker,Albert


In [21]:
UKDict2 = {}

for index, row in UKData2.iterrows():
    a = row['UK_Omeka']
    if a not in UKDict2:
        b= row['First']
        c= row['Last']
        UKDict2.update({a:[b,c]}) 

In [80]:
UKDict2

{33: ['Achenbach', ' Oswald'],
 34: ['Adan', ' Louis-Émile'],
 35: ['Alma Tadema', ' Lawrence'],
 36: ['Anastasi', ' Auguste'],
 37: ['Anker', ' Albert'],
 38: ['Antigna', ' Alexandre'],
 39: ['Appian', ' Adolphe'],
 40: ['Armand-Dumaresq', ' Édouard'],
 41: ['Aubert', ' Jean'],
 42: ['Aublet', ' Albert'],
 43: ['Auguin', ' Louis-Auguste'],
 44: ['Aze', ' Adolphe'],
 45: ['Baader', ' Louis-Marie'],
 46: ['Badin', ' Jules'],
 47: ['Ballavoine', ' Jules'],
 48: ['Balleroy', ' Albert de'],
 49: ['Balze', ' Paul'],
 50: ['Barillot', ' Léon'],
 51: ['Barrias', ' Félix'],
 52: ['Bastien-Lepage', ' Jules'],
 53: ['Baudit', ' Amédée'],
 54: ['Baudry', ' Paul'],
 55: ['Beaucé', ' Jean-Adolphe'],
 56: ['Beaulieu', ' Anatole de'],
 57: ['Beaumetz', ' Étienne'],
 58: ['Beaumont', ' Édouard de'],
 59: ['Beauverie', ' Charles'],
 60: ['Becker', ' Georges'],
 61: ['Bellanger', ' Camille'],
 62: ['Bellay', ' Charles-Paul'],
 63: ['Belly', ' Léon'],
 64: ['Benner', ' Jean'],
 65: ['Bénouville', ' Franç

In [22]:
for index, row in SalonData2.iterrows():
    if ',' in SalonData2.loc[index,'Exposant']:
        a = SalonData2.loc[index,'Exposant'].split(",")
        SalonData2.loc[index,'First'] = a[1]
        SalonData2.loc[index,'Last'] = a[0]
    else:
        SalonData2.loc[index,'Last']=SalonData2.loc[index,'Exposant']

KeyboardInterrupt: 

In [77]:
for index, row in SalonData2.iterrows():
    a= row['Exposant']
    if a in NameDict.keys():
        SalonData2.at[index,'UniqueKey'] = NameDict[a]
        SalonData2.at[index,'Match'] = a
        SalonData2.at[index,'Certainty'] = 100

In [78]:
SalonData2.groupby('Certainty').count()

,Exposant,Titre,Année,Salon,UniqueKey,Match,First,Last
Certainty,,,,,,,,
100,9056,9056,9056,9056,9056,9056,9056,9056
,56388,56388,56388,56388,56388,56388,55845,56388


In [81]:
for index, row in SalonData2.iterrows():
    
# search for fuzzy match >90:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 90 and fuzz.token_sort_ratio(row['First'],value[1]) > 90:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "90-90"
                break

In [82]:
for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 90 and fuzz.token_sort_ratio(row['First'],value[1]) > 85:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "90-85"
                break
for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 90 and fuzz.token_sort_ratio(row['First'],value[1]) > 80:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "90-80"
                break
for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 90 and fuzz.token_sort_ratio(row['First'],value[1]) > 75:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "90-75"
                break

for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 90 and fuzz.token_sort_ratio(row['First'],value[1]) > 70:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "90-70"
                break

In [83]:
SalonData2.groupby('Certainty').count()

,Exposant,Titre,Année,Salon,UniqueKey,Match,First,Last
Certainty,,,,,,,,
100,9056,9056,9056,9056,9056,9056,9056,9056
,53477,53477,53477,53477,53477,53477,52934,53477
90-70,544,544,544,544,544,544,544,544
90-75,340,340,340,340,340,340,340,340
90-80,137,137,137,137,137,137,137,137
90-85,285,285,285,285,285,285,285,285
90-90,1605,1605,1605,1605,1605,1605,1605,1605


In [107]:
Export = SalonData2[(SalonData2['UniqueKey']!='')][['UniqueKey','Match','Exposant','Certainty']].drop_duplicates(subset='Exposant')

In [108]:
Export.to_csv('Other_files/MatchDictionary.csv')

In [101]:
544+340+137+285+1605

2911

In [123]:
for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 85 and fuzz.token_sort_ratio(row['First'],value[1]) > 90:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "85-90"
                break
for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 85 and fuzz.token_sort_ratio(row['First'],value[1]) > 85:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "85-85"
                break
for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 85 and fuzz.token_sort_ratio(row['First'],value[1]) > 80:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "85-80"
                break

for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 85 and fuzz.token_sort_ratio(row['First'],value[1]) > 75:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "85-75"
                break

In [106]:
SalonData2.groupby('Certainty').count()

,Exposant,Titre,Année,Salon,UniqueKey,Match,First,Last
Certainty,,,,,,,,
100,9056,9056,9056,9056,9056,9056,9056,9056
,53209,53209,53209,53209,53209,53209,52666,53209
85-75,16,16,16,16,16,16,16,16
85-80,5,5,5,5,5,5,5,5
85-85,20,20,20,20,20,20,20,20
85-90,227,227,227,227,227,227,227,227
90-70,544,544,544,544,544,544,544,544
90-75,340,340,340,340,340,340,340,340
90-80,137,137,137,137,137,137,137,137


In [124]:
for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 80 and fuzz.token_sort_ratio(row['First'],value[1]) > 90:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "80-90"
                break
for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 80 and fuzz.token_sort_ratio(row['First'],value[1]) > 85:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "80-85"
                break
for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 80 and fuzz.token_sort_ratio(row['First'],value[1]) > 80:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "80-80"
                break

for index, row in SalonData2.iterrows(): 
# search for fuzzy match first name >80:
    if row['UniqueKey']=='': 
        for key, value in UKDict2.items():
            if fuzz.token_sort_ratio(row['Last'],value[0]) > 80 and fuzz.token_sort_ratio(row['First'],value[1]) > 75:
                SalonData2.at[index,'UniqueKey'] = key
                SalonData2.at[index,'Match'] = UKDict[key] 
                SalonData2.at[index,'Certainty'] = "80-75"
                break

In [132]:
Export = SalonData2.groupby(['Certainty','Exposant','Match']).Titre.count()

In [133]:
Export.to_csv('Other_files/MatchDictionary.csv')

In [41]:
matches.head()

,Certainty,Exposant,Match,Titre
0,100,"Achard, Jean-Alexis","Achard, Jean-Alexis",11
1,100,"Achenbach, Oswald","Achenbach, Oswald",13
2,100,"Anastasi, Auguste","Anastasi, Auguste",59
3,100,"Anker, Albert","Anker, Albert",32
4,100,"Antigna, Alexandre","Antigna, Alexandre",17


In [54]:
SalonData2=SalonData2.merge(matches,how='left',on='Exposant')

In [55]:
SalonData2.head()

,Exposant,Titre_x,Année,Salon,UniqueKey,Match_x,Certainty_x,Certainty_y,Match_y,Titre_y
0,"Aarestrup, Mlle Marie","Portrait de Mlle Nilsson, artiste du Théâtre-L...",1866,Salon de 1866,,,,NaN,NaN,1
1,"Aaron, Michel-Joseph",*Giroflées,1878,Salon de 1878,,,,NaN,NaN,1
2,"Abbat, Auguste",Pauvre hirondelle,1880,Salon de 1880,,,,NaN,NaN,1
3,"Abbema, Mlle Louise",*La duchesse Josiane,1875,Salon de 1875,,,,NaN,NaN,9
4,"Abbema, Mlle Louise",*Le déjeûner dans la serre,1877,Salon de 1877,,,,NaN,NaN,2


In [45]:
SalonData2.groupby('Certainty_y').count()

,Exposant,Titre_x,Année,Salon,UniqueKey,Match_x,Certainty_x,Match_y,Titre_y
Certainty_y,,,,,,,,,
100,9056,9056,9056,9056,9056,9056,9056,9056,9056
80-75,32,32,32,32,32,32,32,32,32
85-75,16,16,16,16,16,16,16,16,16
85-80,6,6,6,6,6,6,6,6,6
85-85,20,20,20,20,20,20,20,20,20
85-90,252,252,252,252,252,252,252,252,252
90-70,544,544,544,544,544,544,544,544,544
90-75,340,340,340,340,340,340,340,340,340
90-80,137,137,137,137,137,137,137,137,137


In [56]:
SalonData2.rename(columns={'Titre_x':'Titre','Match_y':'Match','Certainty_y':'Certainty'}, inplace=True)

In [49]:
SalonData2.head()

,Exposant,Titre,Année,Salon,UniqueKey,Match,Certainty
0,"Aarestrup, Mlle Marie","Portrait de Mlle Nilsson, artiste du Théâtre-L...",1866,Salon de 1866,,,NaN
1,"Aaron, Michel-Joseph",*Giroflées,1878,Salon de 1878,,,NaN
2,"Abbat, Auguste",Pauvre hirondelle,1880,Salon de 1880,,,NaN
3,"Abbema, Mlle Louise",*La duchesse Josiane,1875,Salon de 1875,,,NaN
4,"Abbema, Mlle Louise",*Le déjeûner dans la serre,1877,Salon de 1877,,,NaN


In [57]:
SalonData2=SalonData2.drop(['Certainty_x','Match_x','Titre_y'],axis=1)

In [61]:
SalonData2['UniqueKey']= SalonData2['Match'].map(NameDict) 

In [64]:
SalonData2.groupby(['Exposant','Match','UniqueKey']).count()

,,,Titre,Année,Salon,Certainty
Exposant,Match,UniqueKey,,,,
"Achard, Jean-Alexis","Achard, Jean-Alexis",571.0,11,11,11,11
"Achenbach, Osvald","Achenbach, Oswald",33.0,2,2,2,2
"Achenbach, Oswald","Achenbach, Oswald",33.0,13,13,13,13
"Alma-Tadema, Lawrence","Alma Tadema, Lawrence",35.0,1,1,1,1
"Alma-Tadema, Lawrence","Alma Tadema, Lawrence",35.0,1,1,1,1
...,...,...,...,...,...,...
"Ziem, Félix","Ziem, Félix",568.0,37,37,37,37
"Ziégler, Jules","Ziegler, Jules",639.0,3,3,3,3
"Zo, Achille","Zo, Achille",569.0,21,21,21,21


In [65]:
SalonData2.to_csv('Other_files/SalonDatawithKeys.csv')